In [176]:
from selenium import webdriver 
import bs4
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [149]:
wd = webdriver.Chrome()

In [144]:
wd.get('https://www.cricketworldcup.com/teams')

In [145]:
def get_team_urls():
    team_links = []
    team_tag = wd.find_elements(by=By.CSS_SELECTOR, value='.team-index__link')
    team_links = [tag.get_attribute('href') for tag in team_tag]
    return team_links
    

In [146]:
links = get_team_urls()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=118.0.5993.118)
Stacktrace:
	GetHandleVerifier [0x00007FF64A008EF2+54786]
	(No symbol) [0x00007FF649F75612]
	(No symbol) [0x00007FF649E2A64B]
	(No symbol) [0x00007FF649E0A9F5]
	(No symbol) [0x00007FF649E90887]
	(No symbol) [0x00007FF649EA43CF]
	(No symbol) [0x00007FF649E8BC43]
	(No symbol) [0x00007FF649E60941]
	(No symbol) [0x00007FF649E61B84]
	GetHandleVerifier [0x00007FF64A357F52+3524194]
	GetHandleVerifier [0x00007FF64A3AD800+3874576]
	GetHandleVerifier [0x00007FF64A3A5D7F+3843215]
	GetHandleVerifier [0x00007FF64A0A5086+694166]
	(No symbol) [0x00007FF649F80A88]
	(No symbol) [0x00007FF649F7CA94]
	(No symbol) [0x00007FF649F7CBC2]
	(No symbol) [0x00007FF649F6CC83]
	BaseThreadInitThunk [0x00007FFB106C1F87+23]
	RtlUserThreadStart [0x00007FFB125B16B0+32]


In [ ]:
def get_player_urls(team_links):
    player_urls = []
    for link in team_links:
        country = link.split('/')[-1].upper()
        wd.get(link)
        elements = wd.find_elements(by=By.CSS_SELECTOR,value='.player-profile-squad-list__list-item-link')
        for a_tag in elements:
            player_urls.append(a_tag.get_attribute('href'))
        print(f'country { country} completed')
    return player_urls              
    

In [ ]:
player_urls = get_player_urls(links)

country AFGHANISTAN completed
country AUSTRALIA completed
country BANGLADESH completed
country ENGLAND completed
country INDIA completed
country NETHERLANDS completed
country NEW-ZEALAND completed
country PAKISTAN completed
country SOUTH-AFRICA completed
country SRI-LANKA completed


In [177]:
from datetime import datetime
import time


def getPlayerDetails(playerurl):
    wd.get(playerurl)
    time.sleep(3)
    country = playerurl.split('/')[-3].upper()
    dic = {}
    bio = wd.find_element(by=By.CSS_SELECTOR, value='.player-profile-header__info')
    imgurl = bio.find_element(by=By.CSS_SELECTOR, value='.player-profile-header__photo').get_attribute('src')
    name = bio.find_element(by=By.CSS_SELECTOR, value='.player-profile-header__title').text.replace('\n',' ')
    dic['name'] = name
    dic['country'] = country
    dic['imagesource'] = imgurl
    item = [tag.text for tag in bio.find_elements(by=By.CSS_SELECTOR, value='.player-profile-header__meta-text')]
    dic[item[0].split(':')[0].lower()] = item[0].split(':')[1].lower().strip()
    dic[item[1].split(':')[0].lower()] = item[1].split(':')[1].lower().strip()
    if dic['role'] == '-':
        dic['role'] = 'N/A'
    dob = item[2].split(':')[1].lower().strip()
    if dob != '-':
        dt = datetime.strptime(dob, '%d %b %Y')
        formatted_date = dt.strftime('%d-%m-%Y') 
        dic['DateOfBirth'] =  formatted_date
    else :
        dic['DateOfBirth'] = 'N/A'
    dod = item[3].split(':')[1].lower().strip()
    if dod != '-':
        dt = datetime.strptime(dod, '%d %b %Y')
        formatted_debut_date = dt.strftime('%d-%m-%Y') 
        dic['DateOfDebut'] =  formatted_debut_date
    else :
        dic['DateOfDebut'] = 'N/A'
    try :
        batting = wd.find_element(by=By.CSS_SELECTOR, value='.player-rankings-stats__section.player-rankings-stats__section--batting' )
    except NoSuchElementException:
        batting = None
    if batting and 'N/A' not in batting.text :
        text = batting.text.lower().split('\n')
        text = [element.replace(' ','_') for element in text]
        dic[text[0]+'_'+text[1]] = int(text[2])
        dic[text[0]+'_'+text[4]] = int(text[5])
        dic[text[0]+'_'+text[6]] = int(text[7])
        dic[text[0]+'_'+text[6]+'_on'] = text[9].replace('/','-')
        dic[text[0]+'_'+text[10]] = int(text[11])
        dic[text[0]+'_'+text[10]+'_on'] = text[12].replace('/','-')
    else :
        dic['odi_batting_current_ranking'] = 'N/A'
        dic['odi_batting_current_rating'] = 'N/A'
        dic['odi_batting_highest_ranking'] = 'N/A'
        dic['odi_batting_highest_ranking_on'] = 'N/A'
        dic['odi_batting_highest_rating'] = 'N/A'
        dic['odi_batting_highest_rating_on'] = 'N/A'     
    try :
        bowling = wd.find_element(by=By.CSS_SELECTOR, value='.player-rankings-stats__section.player-rankings-stats__section--bowling' )
    except NoSuchElementException:
        bowling = None
    if bowling and 'N/A' not in bowling.text :
        text = bowling.text.lower().split('\n')
        text = [element.replace(' ','_') for element in text]
        dic[text[0]+'_'+text[1]] = int(text[2])
        dic[text[0]+'_'+text[4]] = int(text[5])
        dic[text[0]+'_'+text[6]] = int(text[7])
        dic[text[0]+'_'+text[6]+'_on'] = text[9].replace('/','-')
        dic[text[0]+'_'+text[10]] = int(text[11])
        dic[text[0]+'_'+text[10]+'_on'] = text[12].replace('/','-')
    else :
        dic['odi_bowling_current_ranking'] = 'N/A'
        dic['odi_bowling_current_rating'] = 'N/A'
        dic['odi_bowling_highest_ranking'] = 'N/A'
        dic['odi_bowling_highest_ranking_on'] = 'N/A'
        dic['odi_bowling_highest_rating'] = 'N/A'
        dic['odi_bowling_highest_rating_on'] = 'N/A'  
    try :
        allround = wd.find_element(by=By.CSS_SELECTOR, value='.player-rankings-stats__section.player-rankings-stats__section--allround' )
    except :
        allround = None
    if allround and 'N/A' not in allround.text :
        text = allround.text.lower().split('\n')
        text = [element.replace(' ','_') for element in text]
        dic[text[0]+'_'+text[1]] = int(text[2])
        dic[text[0]+'_'+text[4]] = int(text[5])
        dic[text[0]+'_'+text[6]] = int(text[7])
        dic[text[0]+'_'+text[6]+'_on'] = text[9].replace('/','-')
        dic[text[0]+'_'+text[10]] = int(text[11])
        dic[text[0]+'_'+text[10]+'_on'] = text[12].replace('/','-')
    else :
        dic['odi_allround_current_ranking'] = 'N/A'
        dic['odi_allround_current_rating'] = 'N/A'
        dic['odi_allround_highest_ranking'] = 'N/A'
        dic['odi_allround_highest_ranking_on'] = 'N/A'
        dic['odi_allround_highest_rating'] = 'N/A'
        dic['odi_allround_highest_rating_on'] = 'N/A' 
    return dic


In [180]:
lis = []
for index,player in enumerate(player_urls):
    content = getPlayerDetails(player)
    lis.append(content)
    print(f'Player {index} of name {content["name"]} from country {content["country"]} completed')

Player 0 of name HASHMAT SHAHIDI from country AFGHANISTAN completed
Player 1 of name RAHMAT SHAH from country AFGHANISTAN completed
Player 2 of name NAJIB ZADRAN from country AFGHANISTAN completed
Player 3 of name RASHID KHAN from country AFGHANISTAN completed
Player 4 of name AZMAT OMARZAI from country AFGHANISTAN completed
Player 5 of name REYAZ HASSAN from country AFGHANISTAN completed
Player 6 of name FAZAL FAROOQI from country AFGHANISTAN completed
Player 7 of name NOOR AHMAD from country AFGHANISTAN completed
Player 8 of name MOHAMMAD NABI from country AFGHANISTAN completed
Player 9 of name NAVEEN-UL-HAQ from country AFGHANISTAN completed
Player 10 of name MUJEEB UR RAHMAN from country AFGHANISTAN completed
Player 11 of name IKRAM ALI KHIL from country AFGHANISTAN completed
Player 12 of name IBRAHIM ZADRAN from country AFGHANISTAN completed
Player 13 of name RAHMANULLAH GURBAZ from country AFGHANISTAN completed
Player 14 of name ABDUL RAHMAN from country AFGHANISTAN completed
Pla

In [181]:
import pandas as pd

In [182]:
df = pd.DataFrame(lis)

In [185]:
df.to_json('ICC_WorldCup2023_players.json')

In [186]:
df = df.transpose()

In [187]:
df.to_json('ICC_WC_2023Players.json')

In [193]:
df = df.transpose()
# df = pd.DataFrame(df, index='country')

TypeError: Index(...) must be called with a collection of some kind, 'country' was passed